# Playing around with Glide image model
> The next hurdle tackled?

- toc: false
- branch: master
- badges: true
- comments: true
- author: Jesse van Elteren
- image: images/glide.png
- categories: []

[THIS POST IS WORK IN PROGRESS]
Deep learning has seen spectacular development the past years. OpenAI has been one of the premier organisations publishing spectacular results. Last year their [GPT-3](https://en.wikipedia.org/wiki/GPT-3) was trained on a whopping 500 billion tokens. Then you have a language model which is able to generate all sorts of believable texts.

Now they also released an image model which translates text promts into images. And not like a google search engine that searches existing images, but actually generating new images.
The model is called glide and while they keep the full model for themselves to monatize, they did release a slimmed down version.

I played around with it a bit and let it generate 10 images for each text promt. Below you see the result.


In [172]:
# hide
from pathlib import Path
import imageio as iio
from PIL import Image, ImageDraw, ImageFont
import numpy as np
from functools import reduce

p = Path('images')
filenames = [file.name for file in p.iterdir()]
filenames = [filenames[i].replace('_1.png', '') for i in range(0,len(filenames),10)]

images = list()
for file in Path("images").iterdir():
    im = iio.imread(file)
    images.append(im)

In [173]:
# hide
font = ImageFont.truetype(r'C:\Windows\Fonts\arial.ttf', 20) 
allimgs = []
for i in range(0,len(images),10):
    # concat every 10 images
    imgs = np.hstack(images[i:i+10])
    # add a white box above
    white = np.full((30,2560,3), 255).astype('uint8')
    # convert to image
    text_and_imgs = Image.fromarray(np.concatenate([white, imgs]), 'RGB')
    # add text
    draw = ImageDraw.Draw(text_and_imgs)
    draw.text((3, 5),filenames[i//10], fill='blue', font=font)
    allimgs.append(text_and_imgs)
    

In [1]:
# hide_input
# concat all the text_img files
def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst
full_img = reduce(get_concat_v, allimgs)
full_img.save('full.png')

NameError: name 'reduce' is not defined

![](glide/full.png)

You can clearly see the slimmed down model is not photorealistic. The spaghetti doesn't look very tasty :)
I do find the 'creativity' striking.